In [1]:
!pip install evaluate;
!pip install transformers;
!pip install datasets;

In [2]:
import pandas as pd
import evaluate
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ML_NOAC_NOVA_Extraction_Cleaned_DF3_Full.csv")
data.head()

In [6]:
def make_str(word):
    if not isinstance(word, str):
        return ""
    return str(word)

def introduce_empty(word):
    if not word or not isinstance(word, str) or len(word) == 0:
        return "[EMPTY]"
    return word

def clean_up_pipeline(sentence):
    cleaning_utils = [make_str,
                      #remove_hyperlink,
                      #replace_newline,
                      #to_lower,
                      #remove_number,
                      #remove_punctuation,
                      #remove_whitespace,
                      introduce_empty]
    for o in cleaning_utils:
        sentence = o(sentence)
    return sentence

In [7]:
from transformers import pipeline

In [8]:
pipe = pipeline("sentiment-analysis", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
data["LastEmailContent"] = data["LastEmailContent"].apply(lambda x: clean_up_pipeline(x))

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1, random_state=1)

In [11]:
from datasets import Dataset, DatasetDict

In [12]:
dataset_train = Dataset.from_pandas(train[["LastEmailContent", "Type"]])
dataset_test = Dataset.from_pandas(test[["LastEmailContent", "Type"]])

In [13]:
def tokenize_function(examples):
    return pipe.tokenizer(examples["LastEmailContent"], return_tensors="pt", padding="max_length", truncation=True)

In [14]:
tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_test = dataset_test.map(tokenize_function, batched=True)

  0%|          | 0/227 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

In [15]:
train_test_dataset = DatasetDict({
    'train': tokenized_train,
    'test': tokenized_test})

In [16]:
tokenized_datasets = train_test_dataset.rename_column("Type", "labels")

In [17]:
tokenized_datasets = tokenized_datasets.remove_columns(["LastEmailContent"])

In [18]:
tokenized_datasets.set_format("torch")

In [19]:
#from torch.utils.data import DataLoader
#
#train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
#eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8)

In [20]:
#from torch.optim import AdamW
#
#optimizer = AdamW(pipe.model.parameters(), lr=5e-5)

In [21]:
#from transformers import get_scheduler
#
#num_epochs = 3
#num_training_steps = num_epochs * len(train_dataloader)
#lr_scheduler = get_scheduler(
#    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
#)

In [22]:
#import torch
#
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#pipe.model.to(device);

In [23]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# pipe.model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = pipe.model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

In [24]:
# torch.save(pipe.model, 'model.pth')
# from google.colab import files
# files.download('model.pth')

----------------------------------------------------------------

Reload previous checkpoint

In [25]:
metric = evaluate.load("precision")

In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/test_trainer", evaluation_strategy="epoch")

In [28]:
trainer = Trainer(
    model=pipe.model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

Copy the highest checkpoint listed below in the trainer.train command to restart training at that checkpoint

In [ ]:
!ls "/content/drive/MyDrive/Google Colab - ALab/Model Checkpoints/test_trainer"

checkpoint-68000  runs


In [ ]:
trainer.train("/content/drive/MyDrive/Google Colab - ALab/Model Checkpoints/test_trainer/checkpoint-53000")

In [ ]:
torch.save(pipe.model, 'model.pth')
from google.colab import files
files.download('model.pth')

------------------------------------------------------------

In [30]:
import torch
pipe.model = torch.load("/content/drive/MyDrive/model.pth")

In [36]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25124
  Batch size = 8


{'eval_loss': 0.7003058791160583,
 'eval_precision': 0.31281683001013855,
 'eval_runtime': 470.2552,
 'eval_samples_per_second': 53.426,
 'eval_steps_per_second': 6.679}

In [34]:
output = trainer.predict(tokenized_datasets["test"])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 25124
  Batch size = 8


In [41]:
output

PredictionOutput(predictions=array([[-0.03682597,  0.08888313],
       [-0.00962728,  0.01466707],
       [-0.00120756,  0.00690584],
       ...,
       [-0.04351496,  0.00612719],
       [-0.03476897,  0.04511643],
       [-0.06572963,  0.02878012]], dtype=float32), label_ids=array([1, 0, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.7003058791160583, 'test_precision': 0.31281683001013855, 'test_runtime': 468.0243, 'test_samples_per_second': 53.681, 'test_steps_per_second': 6.711})

In [44]:
import torch.nn.functional as F

y_preds = torch.from_numpy(output.predictions)
probs = F.softmax(y_preds, dim=-1)

In [45]:
for i in probs[0:20] :
  print(i)

tensor([0.4686, 0.5314])
tensor([0.4939, 0.5061])
tensor([0.4980, 0.5020])
tensor([0.4866, 0.5134])
tensor([0.4928, 0.5072])
tensor([0.4920, 0.5080])
tensor([0.4946, 0.5054])
tensor([0.4940, 0.5060])
tensor([0.5004, 0.4996])
tensor([0.4935, 0.5065])
tensor([0.4951, 0.5049])
tensor([0.4920, 0.5080])
tensor([0.4835, 0.5165])
tensor([0.4945, 0.5055])
tensor([0.4908, 0.5092])
tensor([0.4917, 0.5083])
tensor([0.4784, 0.5216])
tensor([0.4935, 0.5065])
tensor([0.4905, 0.5095])
tensor([0.4843, 0.5157])
